In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
cnn = sqlite3.connect('data12.db')
cur = cnn.cursor()

In [3]:
cur.execute('''
CREATE TABLE employee (
    emp_id VARCHAR(2) PRIMARY KEY,
    emp_name VARCHAR(50),
    salary INT,
    dept_id VARCHAR(2),
    manager_id VARCHAR(2),
    FOREIGN KEY (dept_id) REFERENCES department(dept_id),
    FOREIGN KEY (manager_id) REFERENCES manager(manager_id)
);
''')

In [4]:
cur.execute('''
CREATE TABLE department (
    dept_id VARCHAR(2) PRIMARY KEY,
    dept_name VARCHAR(50)
);
''')

In [5]:
cur.execute('''
CREATE TABLE manager (
    manager_id VARCHAR(2) PRIMARY KEY,
    manager_name VARCHAR(50),
    dept_id VARCHAR(2),
    FOREIGN KEY (dept_id) REFERENCES department(dept_id)
);
''')

In [6]:
cur.execute('''
CREATE TABLE projects (
    project_id VARCHAR(2),
    project_name VARCHAR(50),
    team_member_id VARCHAR(2),
    FOREIGN KEY (team_member_id) REFERENCES employee(emp_id),
    FOREIGN KEY (team_member_id) REFERENCES employee(manager_id)
);
''')

In [7]:
cur.execute('''
CREATE TABLE company (
    company_id VARCHAR(4) PRIMARY KEY,
    company_name VARCHAR(50),
    location VARCHAR(50)
);
''')

In [8]:
cur.execute('''
INSERT INTO manager (manager_id, manager_name, dept_id) VALUES
('M1', 'Prem', 'D3'),
('M2', 'Shripadh', 'D4'),
('M3', 'Nick', 'D1'),
('M4', 'Cory', 'D1');
''')

In [9]:
cur.execute('''
INSERT INTO department (dept_id, dept_name) VALUES
('D1', 'IT'),
('D2', 'HR'),
('D3', 'Finance'),
('D4', 'Admin');
''')

In [10]:
cur.execute('''
INSERT INTO employee (emp_id, emp_name, salary, dept_id, manager_id) VALUES
('E1', 'Rahul', 15000, 'D1', 'M1'),
('E2', 'Manoj', 15000, 'D1', 'M1'),
('E3', 'James', 55000, 'D2', 'M2'),
('E4', 'Michael', 25000, 'D2', 'M2'),
('E5', 'Ali', 20000, 'D10', 'M3'),
('E6', 'Robin', 35000, 'D10', 'M3');
''')

In [11]:
cur.execute('''
INSERT INTO projects (project_id, project_name, team_member_id) VALUES
('P1', 'Data Migration', 'E1'),
('P1', 'Data Migration', 'E2'),
('P1', 'Data Migration', 'M3'),
('P2', 'ETL Tool', 'E1'),
('P2', 'ETL Tool', 'M4');
''')

In [12]:
cur.execute('''
INSERT INTO company (company_id, company_name, location) VALUES
('C001', 'TechTFQ Solutions', 'India');
''')

In [13]:
cur.execute('''
SELECT * FROM employee;
''')

result = cur.fetchall()

employee_df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(employee_df)

,emp_id,emp_name,salary,dept_id,manager_id
0,E1,Rahul,15000,D1,M1
1,E2,Manoj,15000,D1,M1
2,E3,James,55000,D2,M2
3,E4,Michael,25000,D2,M2
4,E5,Ali,20000,D10,M3
5,E6,Robin,35000,D10,M3


In [14]:
cur.execute('''
SELECT * FROM department;
''')

result = cur.fetchall()

department_df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(department_df)

,dept_id,dept_name
0,D1,IT
1,D2,HR
2,D3,Finance
3,D4,Admin


In [15]:
cur.execute('''
SELECT * FROM manager;
''')

result = cur.fetchall()

manager_df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(manager_df)

,manager_id,manager_name,dept_id
0,M1,Prem,D3
1,M2,Shripadh,D4
2,M3,Nick,D1
3,M4,Cory,D1


In [16]:
cur.execute('''
SELECT * FROM projects;
''')

result = cur.fetchall()

projects_df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(projects_df)

,project_id,project_name,team_member_id
0,P1,Data Migration,E1
1,P1,Data Migration,E2
2,P1,Data Migration,M3
3,P2,ETL Tool,E1
4,P2,ETL Tool,M4


In [17]:
cur.execute('''
SELECT * FROM company;
''')

result = cur.fetchall()

company_df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(company_df)

,company_id,company_name,location
0,C001,TechTFQ Solutions,India


# INNER JOIN / JOIN

### Fetch Matching Records Only

### Fetch the employee name and the department name they belong to.

In [18]:
cur.execute('''
SELECT e.emp_name, d.dept_name
FROM employee e
INNER JOIN department d on e.dept_id = d.dept_id;
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,emp_name,dept_name
0,Rahul,IT
1,Manoj,IT
2,James,HR
3,Michael,HR


# LEFT JOIN

### LEFT JOIN = Macthing Records + Any Additional Records in the LEFT Table

### Fetch 'All' the employee name and the department name they belong to.

In [19]:
cur.execute('''
SELECT e.emp_name, d.dept_name
FROM employee e
LEFT JOIN department d on e.dept_id = d.dept_id;
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,emp_name,dept_name
0,Rahul,IT
1,Manoj,IT
2,James,HR
3,Michael,HR
4,Ali,None
5,Robin,None


# RIGHT JOIN

### RIGHT JOIN = Macthing Records + Any Additional Records in the RIGHT Table

### Fetch the employee name and 'All' the department name they belong to.

In [20]:
cur.execute('''
SELECT e.emp_name, d.dept_name
FROM employee e
RIGHT JOIN department d on e.dept_id = d.dept_id;
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,emp_name,dept_name
0,Rahul,IT
1,Manoj,IT
2,James,HR
3,Michael,HR
4,None,Finance
5,None,Admin


### Fetch details of All emp, their manager, their department and the projects they work on.

In [21]:
cur.execute('''
SELECT e.emp_name, m.manager_name, d.dept_name, p.project_name
FROM employee e
LEFT JOIN department d ON e.dept_id = d.dept_id
INNER JOIN manager m ON m.manager_id = e.manager_id
LEFT JOIN projects p ON p.team_member_id = e.emp_id
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,emp_name,manager_name,dept_name,project_name
0,Rahul,Prem,IT,Data Migration
1,Rahul,Prem,IT,ETL Tool
2,Manoj,Prem,IT,Data Migration
3,James,Shripadh,HR,None
4,Michael,Shripadh,HR,None
5,Ali,Nick,None,None
6,Robin,Nick,None,None


# FULL JOIN / FULL OUTER JOIN

### FULL JOIN = Macthing Records + All Remaining Records From Left Table & Right Table

### Fetch 'All' the employee name and 'All' the department name they belong to.

In [22]:
cur.execute('''
SELECT e.emp_name, d.dept_name
FROM employee e
FULL JOIN department d ON d.dept_id = e.dept_id;
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,emp_name,dept_name
0,Rahul,IT
1,Manoj,IT
2,James,HR
3,Michael,HR
4,Ali,None
5,Robin,None
6,None,Finance
7,None,Admin


# CROSS JOIN

### Write a query to fetch the employee name and their corresoding department name.
### Also make sure to display the company name and the company location coreesponding to each employee.

In [23]:
cur.execute('''
SELECT e.emp_name, d.dept_name, c.company_name, c.location
FROM employee e
INNER JOIN department d ON d.dept_id = e.dept_id
CROSS JOIN company c;
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,emp_name,dept_name,company_name,location
0,Rahul,IT,TechTFQ Solutions,India
1,Manoj,IT,TechTFQ Solutions,India
2,James,HR,TechTFQ Solutions,India
3,Michael,HR,TechTFQ Solutions,India


# Self Join

In [24]:
cur.execute('''
CREATE TABLE Demo (
    EmployeeID INT PRIMARY KEY,
    EmployeeName VARCHAR(50),
    ManagerID INT
);
''')

In [25]:
cur.execute('''
INSERT INTO Demo (EmployeeID, EmployeeName, ManagerID) VALUES
(1, 'John Doe', NULL),
(2, 'Jane Smith', 1),
(3, 'Bob Johnson', 1),
(4, 'Alice Brown', 2),
(5, 'Charlie Green', 2);
''')

In [26]:
cur.execute('''
SELECT * FROM Demo;
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,EmployeeID,EmployeeName,ManagerID
0,1,John Doe,NaN
1,2,Jane Smith,1.0
2,3,Bob Johnson,1.0
3,4,Alice Brown,2.0
4,5,Charlie Green,2.0


In [28]:
cur.execute('''
SELECT 
    e.EmployeeID AS EmployeeID,
    e.EmployeeName AS EmployeeName,
    m.EmployeeName AS ManagerName
FROM Demo e
JOIN Demo m ON e.ManagerID = m.EmployeeID;
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,EmployeeID,EmployeeName,ManagerName
0,2,Jane Smith,John Doe
1,3,Bob Johnson,John Doe
2,4,Alice Brown,Jane Smith
3,5,Charlie Green,Jane Smith


In [29]:
cur.execute('''
SELECT 
    e.EmployeeID AS EmployeeID,
    e.EmployeeName AS EmployeeName,
    m.EmployeeName AS ManagerName
FROM Demo e
LEFT JOIN Demo m ON e.ManagerID = m.EmployeeID;
''')

result = cur.fetchall()

df = pd.DataFrame(result, columns=[item[0] for item in cur.description])
display(df)

,EmployeeID,EmployeeName,ManagerName
0,1,John Doe,None
1,2,Jane Smith,John Doe
2,3,Bob Johnson,John Doe
3,4,Alice Brown,Jane Smith
4,5,Charlie Green,Jane Smith
